In [1]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
from jieba import posseg
import pickle
import sys
import re
from preprocess_data import *
import logging
import gensim
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
config = config_base.config

In [30]:
df= organize_data(config.train_data)
df = df.sample(5000)
df = deal_data(df)

In [42]:
df_1 = shorten_content_all(df, 500)
df_1 = build_answer_range(df_1)

shorten content, accuracy: 0.9722
max length: 499
min length: 2
mean length:242
median length:225
answer generation accuracy(all): 0.9747
wrong split: 0.0259
answer duplicate: 0.0000


In [40]:
df_2 = shorten_content_fix(df, 500)
df_2 = build_answer_range_fix(df_2)

shorten content, accuracy: 0.9722
max length: 499
min length: 2
mean length:242
median length:225
answer generation accuracy(all): 0.9747
wrong split: 0.0259
answer duplicate: 0.0000


In [41]:
def shorten_content_all(df, max_len):
    """
    :param df:
    :param max_len:
    :return: df
    """
    sys.setrecursionlimit(1000000)
    rouge = Rouge(metrics=['rouge-l'])

    def match(title, content, question, max_len):

        # 如果无法用'。'划分，直接返回 title
        if '。' not in content:
            return title

        def count(flag, content_list):
            """ 查数 """
            number = 0
            for i in range(len(flag)):
                if flag[i] != 0:
                    number += len(content_list[i])+1
            return number

        # 过滤
        content_list = content.split('。')
        temp = []
        for c in content_list:
            if c not in ['', ' ', '  ']:
                temp.append(c)
        content_list = temp
        content_list = [jieba.lcut(c, HMM=False) for c in content_list]
        content_len = len(content_list)

        question_str = ' '.join(jieba.lcut(question, HMM=False))

        # 相似性得分: rouge-l
        scores = []
        for c in content_list:
            if ''.join(c) in question:
                scores.append(-5)
                continue
            c_str = ' '.join(c)
            score = rouge.get_scores(c_str, question_str, avg=True)['rouge-l']['r']
            scores.append(score)

        # 标记类型
        flag = np.zeros(content_len)
        title_number = len(jieba.lcut(title, HMM=False))
        max_len = max_len - title_number
        flag_result = flag.copy()

        # 核心句:
        max_score = max(scores)
        for i in range(content_len):
            if scores[i] == max_score:
                flag[i] = -1
        number = count(flag, content_list)
        if number <= max_len:
            flag_result = flag.copy()
        else:
            temp = []
            c = 0
            for j in range(content_len):
                if flag[j] == -1:
                    c += len(content_list[j]) + 1
                    if c > max_len:
                        break
                    temp.append(''.join(content_list[j]))
            result = [title] + temp

            return '。'.join(result)

        # 核心句下一句
        if number <= max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+1 < content_len) and (flag[i+1] == 0):
                    flag[i+1] = -2
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 最后一句
        if number < max_len:
            if flag[-1] == 0:
                flag[-1] = -3
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 第一句
        if number < max_len:
            if flag[0] == 0:
                flag[0] = -4
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 蕴含句（上+中+下）
        if number < max_len:
            for i in range(content_len):
                if scores[i] == -5:
                    if (i-1 >= 0) and (flag[i-1] == 0):
                        flag[i-1] = -5
                    if flag[i] == 0:
                        flag[i] = -5
                    if (i+1 < content_len) and (flag[i+1] == 0):
                        flag[i+1] = -5
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句下下句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+2 < content_len) and (flag[i+2] == 0):
                    flag[i+2] = -6
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句上一句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i-1 >= 0) and (flag[i-1] == 0):
                    flag[i-1] = -7
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句下下下句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+3 < content_len) and (flag[i+3] == 0):
                    flag[i+3] = -8
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句上上句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i-2 >= 0) and (flag[i-2] == 0):
                    flag[i-2] = -9
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 倒数第二句
        if number < max_len:
            if(len(flag) >= 2) and (flag[-2] == 0):
                flag[-2] = -10
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 第二句
        if number < max_len:
            if (len(flag) >= 2) and (flag[1] == 0):
                flag[1] = -11
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        result = [title]
        for i in range(content_len):
            if flag_result[i] != 0:
                result.append(''.join(content_list[i]))

        # 过滤重复
        temp = []
        for r in result:
            if r not in temp:
                temp.append(r)
        result = temp

        return '。'.join(result)

    titles = df['title'].values
    contents = df['content'].values
    questions = df['question'].values

    merge = [match(t, c, q, max_len) for t, c, q in zip(titles, contents, questions)]
    df['merge'] = merge

    # 评估数据集构建效果
    if 'answer' in df:
        answers = df['answer'].values
        is_in = [True if a in m else False for m, a in zip(merge, answers)]
        df['is_in'] = is_in
        print('shorten content, accuracy: %.4f' % (sum(is_in)/len(df)))

    merge_len = [len(jieba.lcut(m, HMM=False)) for m in merge]
    df['len'] = merge_len
    print('max length: %d' % max(merge_len))
    print('min length: %d' % min(merge_len))
    print('mean length:%d' % df['len'].mean())
    print('median length:%d' % df['len'].median())

    return df

In [39]:
def build_answer_range_fix(df):
    sys.setrecursionlimit(1000000)
    rouge = Rouge(metrics=['rouge-l'])

    def match(merge, answer, question):
        merge_list = jieba.lcut(merge, HMM=False)
        merge_len = len(merge_list)
        answer_list = jieba.lcut(answer, HMM=False)
        answer_len = len(answer_list)
        question_str = ' '.join(jieba.lcut(question, HMM=False))
        start = []
        end = []
        if answer == '':
            return -1, -1
        for i in range(0, merge_len-answer_len+1):
            if merge_list[i: i+answer_len] == answer_list:
                start.append(i)
                end.append(i+answer_len-1)
        if len(start) == 0:
            return -1, -1
        elif len(start) == 1:
            return start[0], end[0]
        else:
            scores = []
            # 前后扩展5个词
            for s, e in zip(start, end):
                s = max(s-5, 0)
                answer_can = ' '.join(merge_list[s: e+5])
                score = rouge.get_scores(answer_can, question_str, avg=True)['rouge-l']['r']
                scores.append(score)
            max_idx = np.argmax(scores)
            return start[max_idx], end[max_idx]

    merges = df[df['is_in']]['merge'].values
    answers = df[df['is_in']]['answer'].values
    questions = df[df['is_in']]['question'].values
    answer_range = [match(m, a, q) for m, a, q in zip(merges, answers, questions)]

    start, end = list(zip(*answer_range))
    df.loc[df['is_in'], 'answer_start'] = start
    df.loc[df['is_in'], 'answer_end'] = end

    merge_len = len(merges)
    right_all_len = (df['answer_end'] >= 0).sum()
    wrong_split_len = (df['answer_end'] == -1).sum()
    wrong_dup_len = (df['answer_end'] == -2).sum()
    print('answer generation accuracy(all): %.4f' % (right_all_len/merge_len))
    print('wrong split: %.4f' % (wrong_split_len/merge_len))
    print('answer duplicate: %.4f' % (wrong_dup_len/merge_len))

    return df

In [ ]:
1